## DNN을 이용한 분류

In [2]:
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd

img = cv2.imread("digits.png", cv2.IMREAD_GRAYSCALE)
print(img.shape)

(1000, 2000)


In [3]:
cells = [np.hsplit(row , 100) for row in np.vsplit(img,50)]
x = np.array(cells) / 255. # 0과 1사이에 하기 위해서 255를 난눠줌. cells 는 0과 255사이에 있기 때문에
X = x[:,:].reshape(-1,400).astype(np.float32)
y= pd.get_dummies(np.repeat(np.arange(10),500)).to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_x , test_x , train_y, test_y = train_test_split(X, y,test_size= 0.3)

In [5]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])

In [6]:
# 어제는 0으로 웨이트를 초기화해줌 
# 이코드는 표준편차가 0.1 인 난수를 발생시켜주세요 라고 하는 것. 

w1 = tf.Variable(tf.truncated_normal([400,512], stddev = 0.1))  
b1 = tf.Variable(tf.constant(0.1, shape = [512]))
h1 = tf.nn.relu(tf.matmul(x,w1)+b1)

In [7]:
w2 =tf.Variable(tf.truncated_normal([512,256], stddev = 0.1))  
b2 =  tf.Variable(tf.constant(0.1, shape = [256]))
h2 = tf.nn.relu(tf.matmul(h1,w2)+b2)

In [8]:
w = tf.Variable(tf.zeros([256,10]))
b = tf.Variable(tf.zeros([10]))

In [9]:
h = tf.nn.softmax(tf.matmul(h2,w)+b)

In [10]:
cross_entropy =-tf.reduce_sum(y*tf.log(h), reduction_indices=[1])

In [11]:
loss = tf.reduce_mean(cross_entropy)  # 손실 함수 

In [16]:
# 0.5 는 학습률을 의미 
train = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [28]:
tot_cnt = len(train_x)

for epoch in range(1000):
    avg_loss = 0
    batch_size = 200
    tot_iter = int(tot_cnt / batch_size)
    for step in range(tot_iter):
        batch_X = train_x[int(step*batch_size):int((step+1)*batch_size)]
        batch_y = train_y[int(step*batch_size):int((step+1)*batch_size)]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if len(train_x)//tot_iter != 0:
        batch_X = train_x[int(tot_iter*batch_size):]
        batch_y = train_y[int(tot_iter*batch_size):]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if epoch%100 ==0:
        print("step {}, train loss {}".format(epoch, avg_loss))


step 0, train loss 2.087545116784681e-05
step 100, train loss 1.996832920400159e-05
step 200, train loss 1.913346846670131e-05
step 300, train loss 1.8360909641213604e-05
step 400, train loss 1.764402446390116e-05
step 500, train loss 1.6979918227126988e-05
step 600, train loss 1.6359490098907638e-05
step 700, train loss 1.5781950798341044e-05
step 800, train loss 1.5241040781641939e-05
step 900, train loss 1.4732621954916631e-05


In [26]:
# 예측해보기
correct_prediction = tf.equal(tf.argmax(h,1), tf.argmax(y,1)) # 1 은 축을 의미함
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_x,y:test_y}))

0.944


In [27]:
print(sess.run(h, feed_dict={x:test_x})[0:5])

[[3.60679151e-05 3.12727647e-14 1.92031008e-03 1.08969934e-11
  2.64617213e-12 1.37196015e-08 9.98043537e-01 3.67975615e-13
  2.13918249e-12 1.17265878e-15]
 [1.17145821e-10 9.99995947e-01 5.72371164e-07 7.28954050e-11
  3.33894086e-08 1.48374868e-10 2.30492310e-06 1.69613195e-08
  1.23087352e-06 4.68823250e-14]
 [2.97887088e-16 1.50920130e-08 3.32287149e-13 3.61405350e-11
  9.99946356e-01 2.84389751e-11 3.63215014e-12 1.52481927e-09
  5.35904292e-05 5.09354736e-09]
 [1.62699577e-17 1.05216552e-14 2.38997546e-14 7.30494936e-15
  1.00000000e+00 3.78292997e-09 1.91704161e-12 2.01976090e-13
  2.64013394e-10 2.18059544e-08]
 [1.11491560e-09 5.29078825e-08 4.65063116e-04 1.11223508e-09
  7.30984651e-09 1.36111344e-08 9.99534249e-01 1.26094718e-10
  5.61115144e-07 1.55170096e-11]]


- 그래딘티 베니싱 ? 문제 해결하려면
- 배치 정규화
- 웨이트를 0,1, 이아닌 다른 것을 초기화한다 

In [23]:
predict_y = sess.run(tf.argmax(h,1),feed_dict={x:test_x})
print(predict_y)

[6 1 4 ... 3 8 9]


In [24]:
print(test_y.argmax(axis=1)[0:20])

[2 1 4 4 6 4 7 8 9 6 1 3 5 5 0 0 7 0 9 1]


In [22]:
predict_y = sess.run(tf.argmax(h,1), feed_dict={x:test_x})
cross_tab= pd.crosstab(test_y.argmax(axis=1),predict_y,margins= True)
cross_tab

col_0,0,1,2,3,4,5,6,7,8,9,All
row_0,,,,,,,,,,,
0,142,0,0,1,0,1,0,0,0,0,144
1,0,150,4,1,0,0,0,1,0,0,156
2,1,0,122,0,1,0,1,1,2,1,129
3,0,1,3,136,1,2,1,2,2,2,150
4,1,0,0,0,141,0,1,1,1,6,151
5,0,0,1,5,0,150,6,0,1,0,163
6,1,0,1,0,1,2,141,0,0,0,146
7,0,0,2,0,1,0,0,143,0,1,147
8,0,0,2,6,1,4,2,0,151,0,166


## CNN을 이용한 딥러닝

In [29]:
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd

img = cv2.imread("digits.png", cv2.IMREAD_GRAYSCALE)
print(img.shape)

(1000, 2000)


In [30]:
cells = [np.hsplit(row , 100) for row in np.vsplit(img,50)]
x = np.array(cells) / 255. # 0과 1사이에 하기 위해서 255를 난눠줌. cells 는 0과 255사이에 있기 때문에
X = x[:,:].reshape(-1,400).astype(np.float32)
y= pd.get_dummies(np.repeat(np.arange(10),500)).to_numpy()

### 2.2 레이어 정의

In [31]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1,20,20,1])

In [36]:
w_conv1 = tf.Variable(tf.truncated_normal([3,3,1,32], stddev= 0.1))
# b_conv1  = tf.constant(0.1,shape=[32])

h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, w_conv1, strides = [1,1,1,1] , padding='SAME'))
h_pool1 = tf.nn.max_pool(h_conv1, ksize = [1,2,2,1], strides = [1,2,2,1] , padding='SAME')
h_pool1_drop = tf.nn.dropout(h_pool1, rate=0.1)

In [57]:
w_conv2 = tf.Variable(tf.truncated_normal([3,3,32,64], stddev= 0.1))
# b_conv1  = tf.constant(0.1,shape=[32])

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1_drop, w_conv2, strides = [1,1,1,1] , padding='SAME'))
h_pool2 = tf.nn.max_pool(h_conv2, ksize = [1,2,2,1], strides = [1,2,2,1] , padding='SAME')
h_pool2_drop = tf.nn.dropout(h_pool2, rate=0.25)

In [60]:
W_fc1 = tf.Variable(tf.truncated_normal([5*5*64, 1024], stddev=0.1)) # 5*5는 이미지 크기에 따라 바뀐다.
h_pool2_flat = tf.reshape(h_pool2, [-1,5*5*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1))
h_fc1_drop = tf.nn.dropout(h_fc1, rate=0.5)

W_fc2 = tf.Variable(tf.truncated_normal([1024,10], stddev=0.1))
h = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2))

cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer(0.001).minimize(loss)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

tot_cnt = len(train_x)

In [61]:
for epoch in range(20):
    avg_loss = 0
    batch_size = 128
    tot_iter = int(tot_cnt / batch_size)
    for step in range(tot_iter):
        batch_x = train_x[int(step*batch_size):int((step+1)*batch_size)]
        batch_y = train_y[int(step*batch_size):int((step+1)*batch_size)]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_x, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if len(train_x)//tot_iter != 0:
        batch_x = train_x[int(tot_iter*batch_size):]
        batch_y = train_y[int(tot_iter*batch_size):]
        print("{:03d} - {:f}".format(step, loss_), end="\r")

In [62]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_x, y:test_y}))

0.968


## 케라스를 이용한 분류 (CNN) 

###  입력데이터

In [54]:
import cv2
import pandas as pd
img = cv2.imread("digits.png", cv2.IMREAD_GRAYSCALE)

In [55]:
cells = [np.hsplit(row,100) for row in np.vsplit(img,50)]
x = np.array(cells)

In [56]:
cells = [np.hsplit(row,100) for row in np.vsplit(img,50)]
x = np.array(cells) 
img_rows, img_cols = 20,20
x = x.reshape(-1, img_rows,img_cols, 1)
y = pd.get_dummies(np.repeat(np.arange(10),500)).to_numpy()

In [57]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y , test_y = train_test_split(x,y,test_size=0.3 , 
                                                    random_state=1)

### 이미지 파일에 저장하기

In [58]:
import os
path = "digits"
for i, img_data in enumerate(x):
    number_dir = str(y[i])
    img_name = '%04d.jpg'%i
    img_dir = os.path.join(path, number_dir)
    img_path = os.path.join(path, number_dir, img_name)
    os.makedirs(img_dir, exist_ok=True)
    cv2.imwrite(img_path, img_data)
    # 아래 코드는 진행상황을 보기 위함
    percent = (i/5000) * 100 # 숫자 5000개
    progress = int(percent*5/10) * '#'
    print("{:5.1f}%[{:<50s}] {}".format(percent, progress, i), end= "\r")
    # if i>2:
#         break:

  7.4%[###                                               ] 371

### CNN 레이어 정의

In [59]:
from tensorflow.keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
model.add(Conv2D(32, kernel_size = (3,3),
                input_shape= (img_rows, img_cols , 1 ), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))
          
          

In [60]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = "digits-{epoch:03d}-{val_acc:.4f}.hdf5",
                            monitor="val_acc", save_best_only=True, verbose=1)

In [61]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_acc", patience = 5)

In [62]:
import tensorflow as tf
model.compile(loss= tf.keras.losses.categorical_crossentropy,
             optimizer = tf.keras.optimizers.Adam(0.001),
             metrics= ["accuracy"])

In [63]:
model.fit(train_x, train_y, validation_data = (test_x, test_y),
         callbacks=[checkpoint, early_stopping],
         batch_size=128, epochs=50, verbose=1)

Train on 3500 samples, validate on 1500 samples
Epoch 1/50
3456/3500 [============================>.] - ETA: 0s - loss: 17.3215 - acc: 0.2127
Epoch 00001: val_acc improved from -inf to 0.33600, saving model to digits-001-0.3360.hdf5
3500/3500 [==============================] - 1s 282us/sample - loss: 17.1374 - acc: 0.2146 - val_loss: 1.9468 - val_acc: 0.3360
Epoch 2/50
3328/3500 [===========================>..] - ETA: 0s - loss: 2.1773 - acc: 0.2437
Epoch 00002: val_acc improved from 0.33600 to 0.37533, saving model to digits-002-0.3753.hdf5
3500/3500 [==============================] - 1s 200us/sample - loss: 2.1688 - acc: 0.2460 - val_loss: 1.8831 - val_acc: 0.3753
Epoch 3/50
3200/3500 [==========================>...] - ETA: 0s - loss: 1.9742 - acc: 0.2859
Epoch 00003: val_acc improved from 0.37533 to 0.55667, saving model to digits-003-0.5567.hdf5
3500/3500 [==============================] - 1s 205us/sample - loss: 1.9695 - acc: 0.2931 - val_loss: 1.5304 - val_acc: 0.5567
Epoch 4/50


3328/3500 [===========================>..] - ETA: 0s - loss: 0.2883 - acc: 0.9123
Epoch 00028: val_acc did not improve from 0.95333
3500/3500 [==============================] - 1s 192us/sample - loss: 0.2895 - acc: 0.9117 - val_loss: 0.1502 - val_acc: 0.9513
Epoch 29/50
3456/3500 [============================>.] - ETA: 0s - loss: 0.2734 - acc: 0.9164- ETA: 0s - loss: 0.2638 - acc: 0
Epoch 00029: val_acc improved from 0.95333 to 0.95400, saving model to digits-029-0.9540.hdf5
3500/3500 [==============================] - 1s 200us/sample - loss: 0.2727 - acc: 0.9163 - val_loss: 0.1610 - val_acc: 0.9540
Epoch 30/50
3456/3500 [============================>.] - ETA: 0s - loss: 0.2428 - acc: 0.9207
Epoch 00030: val_acc did not improve from 0.95400
3500/3500 [==============================] - 1s 191us/sample - loss: 0.2419 - acc: 0.9214 - val_loss: 0.1509 - val_acc: 0.9487
Epoch 31/50
3200/3500 [==========================>...] - ETA: 0s - loss: 0.2348 - acc: 0.9272
Epoch 00031: val_acc did not

In [64]:
model_json = model.to_json()
with open("digits_model.json","w") as json_file:
    json_file.write(model_json)

In [65]:
from tensorflow.keras.models import model_from_json
with open("digits_model.json","r") as json_file:
    loaded_model_json =json_file.read()
    model = model_from_json(loaded_model_json)

In [67]:
model.load_weights("digits-044-0.9667.hdf5")

In [68]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=["accuracy"])
model.evaluate(test_x, test_y, verbose =0)

[0.1219771695608894, 0.96666664]

In [69]:
import cv2
img4 = cv2.imread("four.png",0)
number = img4.reshape(-1, img_rows, img_cols, 1)

In [70]:
np.argmax(model.predict(number))

6

### 웹카메라로 숫자 인식하기

In [71]:
import cv2
cap = cv2.VideoCapture(0)
if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 110,225, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                                   cv2.CHAIN_APPROX_SIMPLE)
            try:
                for i in range(len(contours)):
                    contour = contours[i]
                    (x,y), radius = cv2.minEnclosingCircle(contour)
                    if radius > 3:
                        xs, xe = int(x-radius), int(x+radius)
                        ys,ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img,(xs, ys),(xe,ye),
                                     (200,0,0),1)
                        roi = bin_img[ys:ye, xs:xe]
                        dst = cv2.resize(roi, dsize = (50,50), interpolation=cv2.INTER_AREA)
                        dst = cv2.resize(roi, dsize = (16,16), interpolation=cv2.INTER_AREA)
                        A = np.zeros((20,20))
                        A[2:-2,2:-2] = dst[:,:]
                        A = A.reshape(-1,20,20,1)
                        num = model.predict(A)
                        cv2.putTextt(bin_img, str(num),(xs, ys), cv2.FONT_HERSHEY_PLAIN,2.(200,0,0))
            except Exception as e:
                pass
            cv2.imshow("Image",bin_img)
            if cv2.waitKey(1)&0xFF == 27 : # ESC
                break
        else:
            print("no frame")
            break
else:
    print("camera not opened")
    
cap.release()
cv2.destroyAllWindows()

camera not opened


In [72]:
32

32

### 이미지 증강

In [78]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2,
                                  height_shift_range=0.2, 
                                  shear_range=0.2, zoom_range= 0.2,
                                  horizontal_flip = True,
                                  fill_mode="nearest")

test_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2,
                                  height_shift_range=0.2, 
                                  shear_range=0.2, zoom_range= 0.2,
                                  horizontal_flip = True,
                                  fill_mode="nearest")

In [ ]:
import cv2
img0 = cv2.imread("digits/1/1000.jpg",0)
x =img0.reshape(-1,20,20,1)

for batch in train_datagen.flow(x, batch_size=1,
                               save_to_dir="digits",save_prefix="zero",
                               save_format="jpeg"):
    i+=1
    if i >20 :
        break

In [82]:
from tensorflow.keras.models import model_from_json
with open("digits_model.json","r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)

In [83]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = "digits-{epoch:03d}-{val_acc:.4f}.hdf5",
                            monitor="val_acc", save_best_only=True, verbose=1)

In [84]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_acc", patience = 5)

In [85]:
import tensorflow as tf
model.compile(loss= tf.keras.losses.categorical_crossentropy,
             optimizer = tf.keras.optimizers.Adam(0.001),
             metrics= ["accuracy"])

In [89]:
model.fit(train_datagen.flow(train_x, train_y),
          # steps_per_epoch= len(trian_x) / 128,
          validation_data = test_datagen.flow(test_x, test_y),
         callbacks=[checkpoint, early_stopping],
         batch_size=500,verbose=1)

105/110 [===========================>..] - ETA: 0s - loss: 2.2787 - acc: 0.1470
Epoch 00001: val_acc did not improve from 0.83400
110/110 [==============================] - 1s 12ms/step - loss: 2.2771 - acc: 0.1474 - val_loss: 2.1986 - val_acc: 0.1647


In [ ]:
import cv2
cap = cv2.VideoCapture(0)
if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 110,225, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                                   cv2.CHAIN_APPROX_SIMPLE)
            try:
                for i in range(len(contours)):
                    contour = contours[i]
                    (x,y), radius = cv2.minEnclosingCircle(contour)
                    if radius > 3:
                        xs, xe = int(x-radius), int(x+radius)
                        ys,ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img,(xs, ys),(xe,ye),
                                     (200,0,0),1)
                        roi = bin_img[ys:ye, xs:xe]
                        dst = cv2.resize(roi, dsize = (50,50), interpolation=cv2.INTER_AREA)
                        dst = cv2.resize(roi, dsize = (16,16), interpolation=cv2.INTER_AREA)
                        A = np.zeros((20,20))
                        A[2:-2,2:-2] = dst[:,:]
                        A = A.reshape(-1,20,20,1)
                        num = model.predict(A)
                        cv2.putTextt(bin_img, str(num),(xs, ys), cv2.FONT_HERSHEY_PLAIN,2.(200,0,0))
            except Exception as e:
                pass
            cv2.imshow("Image",bin_img)
            if cv2.waitKey(1)&0xFF == 27 : # ESC
                break
        else:
            print("no frame")
            break
else:
    print("camera not opened")
    
cap.release()
cv2.destroyAllWindows()